In [72]:
import torch
from lm_from_scratch.corpus.decision_corpus import DecisionCorpus
from lm_from_scratch.tokenizers.char_tokenizer import CharLevelTokenizer
# from lm_from_scratch.models.gpt_ma import Gpt
from lm_from_scratch.models.gpt_ma_2 import Gpt

BATCH_SIZE = 64 # how many independent sequences will we process in parallel?
MAX_LEN = 128 # what is the maximum context length for predictions?
MAX_ITERS = 10000
EVAL_INTERVAL = 1000
LEARNING_RATE = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EVAL_ITERS = 200

VOCAB_SIZE = 10000
N_EMBEDDING = 384
NUM_HEAD = 6

# NUM_HEAD * HEAD_SIZE = N_EMBEDDING
HEAD_SIZE = N_EMBEDDING // NUM_HEAD

In [73]:
corpus = DecisionCorpus()

corpus_df = corpus.df.sample(
        frac=1,
        random_state=42
    ).reset_index(
        drop=True)

# Train and test splits
n = int(0.9*len(corpus_df)) # first 90% will be train, rest val

data = corpus.get_text()

In [74]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=["[UNK]"])
tokenizer.pre_tokenizer = Whitespace()

tokenizer.train_from_iterator(data, trainer)

# tokenizer = CharLevelTokenizer("".join(data))

In [75]:
corpus_df["idx"] = [torch.tensor(text.ids, dtype=torch.long) for text in tokenizer.encode_batch(data)]

train_data = corpus_df["idx"][:n]
val_data = corpus_df["idx"][n:].reset_index(drop=True)

In [1]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data

    doc_ix = torch.randint(len(data), (BATCH_SIZE,))
    doc_token_ix = []

    for doc_i in doc_ix:
        ix = torch.randint(len(data[int(doc_i)]) - MAX_LEN, (1,))
        doc_token_ix.append((int(doc_i), ix))
    
    x = torch.stack([data[i][j:j+MAX_LEN] for i,j in doc_token_ix])
    y = torch.stack([data[i][j+1:j+MAX_LEN+1] for i,j in doc_token_ix])

    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y


In [77]:
model = Gpt(VOCAB_SIZE, NUM_HEAD, HEAD_SIZE, N_EMBEDDING, max_len=MAX_LEN)
m = model.to(DEVICE)

In [78]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

for iter in range(MAX_ITERS):

    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVAL == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 9.3776, val loss 9.3771
step 1000: train loss 1.4585, val loss 2.2623
step 2000: train loss 1.2907, val loss 2.1093
step 3000: train loss 1.1868, val loss 2.0281
step 4000: train loss 1.1159, val loss 1.9703
step 5000: train loss 1.0875, val loss 1.9756
step 6000: train loss 1.0274, val loss 1.9573
step 7000: train loss 1.0134, val loss 1.9673
step 8000: train loss 0.9722, val loss 1.9424
step 9000: train loss 0.9581, val loss 1.9390
step 10000: train loss 0.9409, val loss 1.9492
step 11000: train loss 0.9191, val loss 1.9622
step 12000: train loss 0.9082, val loss 1.9717
step 13000: train loss 0.8962, val loss 1.9879
step 14000: train loss 0.8743, val loss 1.9864


In [88]:
from torch.nn import functional as F

# generate from the model
def generate(model, idx, max_new_tokens, max_len):
    # idx is (B, T) array of indices in the current context
    for _ in range(max_new_tokens):
        # crop idx to the last max_len tokens
        idx_cond = idx[:, -max_len:]
        # get the predictions
        logits, loss = model(idx_cond)
        # focus only on the last time step
        logits = logits[:, -1, :] # becomes (B, C)
        # apply softmax to get probabilities
        probs = F.softmax(logits, dim=-1) # (B, C)
        # sample from the distribution
        idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
        # append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
    return idx

context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)

context = torch.tensor(
            tokenizer.encode("COUR DE CASSATION\n Premi\u00e8re pr\u00e9sidence").ids,
            dtype=torch.long,
            device=DEVICE)

print(tokenizer.decode(generate(model,
                          context[None, :],
                          max_len=MAX_LEN,
                          max_new_tokens=100)[0].tolist()))


COUR DE CASSATION Première présidence __________ [ F ] Pourvoi n ° : K 23 - 11 . 155 Demandeur ( s ) : M . [ S ] Avocat ( s ) : la SARL Boré , Salve de Bruneton et Mégret Défendeur ( s ) : Mme [ E ] et autre Avocat ( s ) : la SCP Célice , Texidor , Périer Ordonnance : 50 510 ORDONNANCE DE DÉCHÉANCE Mme Caroline Azar , conseillère référendaire , déléguée par le premier président de la Cour de cassation , a rendu la présente ordonnance . M . [ H ] [ B ],
